In [1]:
from syft_rds.orchestra import setup_rds_stack
from syft_rds import RDS_NOTEBOOKS_PATH

In [2]:
from syft_core.url import SyftBoxURL


In [3]:
SyftBoxURL

syft_core.url.SyftBoxURL

In [4]:
stack = setup_rds_stack(log_level="INFO")

2025-03-07 12:46:47.581 | WARNING  | syft_rds.orchestra:setup_rds_stack:89 - Using shared data directory for both clients. Any file permission checks will be skipped as both clients have access to the same files.
2025-03-07 12:46:47.593 | INFO     | syft_rds.orchestra:setup_rds_stack:112 - Launching mock RDS stack in /var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/rds_1iouxw3n...


In [5]:
do_client = stack.do_rds_client
ds_client = stack.ds_rds_client

# Create a dataset as DO

In [6]:
CWD = RDS_NOTEBOOKS_PATH / "quickstart"

In [7]:
private_dir = CWD / "data" / "dataset-1" / "private"
mock_dir = CWD / "data" / "dataset-1" / "mock"
markdown_path = CWD / "data" / "dataset-1" / "description.md"

private_dir.mkdir(parents=True, exist_ok=True)
mock_dir.mkdir(parents=True, exist_ok=True)


with open(private_dir / "data.csv", "w") as f:
    f.write("-1,-2,-3")

with open(mock_dir / "data.csv", "w") as f:
    f.write("1,2,3")

with open(markdown_path, "w") as f:
    f.write("some description")


In [8]:
data = do_client.dataset.create(
    name="dataset-1",  # MUST BE UNIQUE. Throw Exception if already exist.
    path=private_dir,  # MUST EXIST
    mock_path=mock_dir,
    summary="dummy data",
    description_path=markdown_path,
)

In [9]:
do_client.datasets

[Dataset
   uid: 07d47774-9f37-4db5-a9f0-588c87e6cd1f
   created_at: 2025-03-07T11:46:47.812700Z
   updated_at: 2025-03-07T11:46:47.812706Z
   client_id: None
   name: dataset-1
   private: syft://data_owner@test.openmined.org/private/datasets/dataset-1
   mock: syft://data_owner@test.openmined.org/public/datasets/dataset-1
   summary: dummy data
   readme: syft://data_owner@test.openmined.org/public/datasets/dataset-1/description.md
   tags: []]

# Use dataset as DS

In [10]:
ds_client.datasets

[Dataset
   uid: 07d47774-9f37-4db5-a9f0-588c87e6cd1f
   created_at: 2025-03-07T11:46:47.812700Z
   updated_at: 2025-03-07T11:46:47.812706Z
   client_id: None
   name: dataset-1
   private: syft://data_owner@test.openmined.org/private/datasets/dataset-1
   mock: syft://data_owner@test.openmined.org/public/datasets/dataset-1
   summary: dummy data
   readme: syft://data_owner@test.openmined.org/public/datasets/dataset-1/description.md
   tags: []]

In [11]:
ds_client.dataset.get("dataset-1").get_mock_path()


PosixPath('/private/var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/rds_1iouxw3n/shared_client_dir/datasites/data_owner@test.openmined.org/public/datasets/dataset-1')

In [12]:
ds_client.dataset.get("dataset-1").get_private_path()

PosixPath('/private/var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/rds_1iouxw3n/shared_client_dir/datasites/data_owner@test.openmined.org/private/datasets/dataset-1')

In [13]:
dataset = ds_client.dataset.get("dataset-1")
dataset.describe()

├── data.csv
└── description.md


True

In [14]:
import pathlib
from syft_rds.models.models import Dataset
from syft_rds.utils.render_code_template import render_data_access_template

In [15]:
my_file = CWD / "data" / "code1.py"

In [16]:
%%writefile {my_file}

import os

DATA_DIR = os.environ["DATA_DIR"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]

print("Hello, world!")

with open(os.path.join(OUTPUT_DIR, "output.txt"), "w") as f:
    f.write("ABC")


Overwriting /Users/koen/workspace/rds/notebooks/quickstart/data/code1.py


In [17]:
job = ds_client.jobs.submit(
    user_code_path=my_file,
    dataset_name = dataset.name
)
job


Job
  uid: 1cc8479e-224e-4737-93e0-2143ec6d9057
  created_at: 2025-03-07T11:46:48.343335Z
  updated_at: 2025-03-07T11:46:48.343338Z
  client_id: 24a8c561-b1d0-40e7-8c10-21c01fa9062f
  name: binary_signal_0ldj
  description: None
  user_code_id: 5f7bb945-b971-4c03-82a7-c816d5b33b80
  tags: []
  user_metadata: {}
  status: pending_code_review
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/1cc8479e-224e-4737-93e0-2143ec6d9057
  dataset_name: dataset-1

In [18]:
ds_client

In [19]:
jobs = do_client.jobs.get_all()

In [20]:
job = jobs[-1]

In [21]:
job

Job
  uid: 1cc8479e-224e-4737-93e0-2143ec6d9057
  created_at: 2025-03-07T11:46:48.343335Z
  updated_at: 2025-03-07T11:46:48.343338Z
  client_id: 57af69e3-46fa-4226-8fac-acfe13423fa4
  name: binary_signal_0ldj
  description: None
  user_code_id: 5f7bb945-b971-4c03-82a7-c816d5b33b80
  tags: []
  user_metadata: {}
  status: pending_code_review
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/1cc8479e-224e-4737-93e0-2143ec6d9057
  dataset_name: dataset-1

In [22]:
res_job = do_client.run_private(job)

2025-03-07 12:46:48.764 | WARNING  | syft_rds.client.rds_client:run_private:123 - Running job without docker is not secure


╭───────────────────────────── Job Configuration ──────────────────────────────╮
│ Starting job                                                                 │
│ Execution: python code1.py                                                   │
│ Dataset Dir.:  .../data_owner@test.openmined.org/private/datasets/dataset-1  │
│ Output Dir.:   .../tmp/syft-rds-jobs/1cc8479e224e473793e02143ec6d9057/output │
│ Timeout:  60s                                                                │
╰──────────────────────────────────────────────────────────────────────────────╯

Running job...

Hello, world!

Job completed successfully!

In [23]:
do_client.jobs.share_results(res_job);

2025-03-07 12:46:49.133 | INFO     | syft_rds.client.rds_clients.jobs:share_results:94 - Shared results for job 1cc8479e-224e-4737-93e0-2143ec6d9057 at /private/var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/rds_1iouxw3n/shared_client_dir/datasites/data_owner@test.openmined.org/api_data/RDS/output/1cc8479e-224e-4737-93e0-2143ec6d9057


In [24]:
from syft_rds.models.models import GetAllRequest

In [25]:
#job.add_to_queue()

In [26]:
job = ds_client.jobs.get_all()[-1]

In [27]:
job._syftbox_client

In [28]:
ds_client._syftbox_client.datasites

PosixPath('/private/var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/rds_1iouxw3n/shared_client_dir/datasites')

In [29]:
job.output_url

'syft://data_owner@test.openmined.org/api_data/RDS/output/1cc8479e-224e-4737-93e0-2143ec6d9057'

In [30]:
job.get_output_path()

PosixPath('/private/var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/rds_1iouxw3n/shared_client_dir/datasites/data_owner@test.openmined.org/api_data/RDS/output/1cc8479e-224e-4737-93e0-2143ec6d9057')